In [1]:
import numpy as np
import pandas as pd

In [11]:
# Define the range of wind speeds and the number of states
wind_speed_range = (0, 50)
wind_speed_states = 50
wind_speed_interval = (wind_speed_range[1] - wind_speed_range[0]) / wind_speed_states
wind_speed_states_list = [wind_speed_range[0] + i * wind_speed_interval for i in range(wind_speed_states)]

# Define the range of angles of attack and the number of states
angle_range = (-15, 15)
angle_states = 30
angle_interval = (angle_range[1] - angle_range[0]) / angle_states
angle_states_list = [angle_range[0] + i * angle_interval for i in range(angle_states)]

# Define the number of actions
actions = 3
increase_angle = 0
decrease_angle = 1
keep_angle = 2

#define wind speed change
wind_speed_change = 1

counter = 0 #counts the number of times it takes for algorithm to keep the lift constant within loop

In [16]:

# Initialize the table
table = []
headers = ['Wind Speed', 'Angle', 'Lift', 'Reward', 'counter']
table.append(headers)


# Define the Q-table and some initial parameters
q_table = np.zeros((wind_speed_states, angle_states, actions))
alpha = 0.1
gamma = 0.9
epsilon = 0.1
# maximum number of episodes
max_episodes = 1000

# Define the function to calculate the lift of the airfoil
def lift(angle_of_attack, wind_speed):
    # Calculate the Reynolds number
    air_density = 1.225 # kg/m^3 (sea level)
    air_viscosity = 1.78e-5 # Ns/m^2 (sea level)
    chord_length = 0.15 # m (geometry of the NACA 0015 airfoil)
    reynolds_number = wind_speed * chord_length / air_viscosity
    
    # Calculate the lift coefficient
    C_l_alpha = 2 * np.pi / 180
    C_l_0 = 0.0060
    lift_coefficient = C_l_alpha * angle_of_attack + C_l_0
    
    # Calculate the lift force
    lift_force = 0.5 * lift_coefficient * air_density * wind_speed ** 2 * chord_length
    
    return lift_force

# Define the Q-learning algorithm
for episode in range(max_episodes):
    # Initialize the state and the lift
    current_wind_speed = 20 # m/s
    current_angle = 15 # degrees
    current_lift = lift(current_angle, current_wind_speed)
    counter = 0
    # Run the Q-learning algorithm
    while True:
        counter += 1
        # Choose an action based on the Q-table and epsilon-greedy strategy
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.randint(0, actions)
        else:
            action = np.argmax(q_table[current_wind_speed][current_angle])
        
        # Calculate the new lift and the new state
        new_angle = current_angle + action
        if new_angle < angle_range[0]:
             new_angle = angle_range[0]
        elif new_angle >= angle_range[1]:
             new_angle = angle_range[1] -1
                
        new_wind_speed = current_wind_speed + wind_speed_change

        # Check if the new wind speed is within the range of possible wind speeds
        if new_wind_speed < wind_speed_range[0]:
            new_wind_speed = wind_speed_range[0]
        elif new_wind_speed >= wind_speed_range[1]:
            new_wind_speed = wind_speed_range[1] -1

        new_lift = lift(new_angle, new_wind_speed)
        current_lift = lift(current_angle, current_wind_speed)
        
        # Define the lift threshold
        lift_threshold = 0.01 # 1% of the maximum lift force
        reward = -1 if abs(new_lift - current_lift) > lift_threshold else 1
        
        # Last check for windspeed and angle
        if new_wind_speed >= wind_speed_states:
            new_wind_speed = wind_speed_states - 1
        elif new_wind_speed < 0:
            new_wind_speed = 0
    
        if new_angle >= angle_states:
            new_angle = angle_states - 1
        elif new_angle < 0:
            new_angle = 0
            
        # Update the Q-table
        q_table[current_wind_speed][current_angle][action] = (1 - alpha) * q_table[current_wind_speed][current_angle][action] + alpha * (reward + gamma * np.max(q_table[new_wind_speed][new_angle])) 
        
        # Check if the lift is constant
        if abs(new_lift - current_lift) < lift_threshold:
            break
    
        # Update the current state and lift
        current_wind_speed = new_wind_speed
        current_angle = new_angle
        current_lift = new_lift    
                
    if episode % 10 == 0:
            table.append([current_wind_speed, current_angle, current_lift, reward, counter])
        
        
        


In [17]:
table

[['Wind Speed', 'Angle', 'Lift', 'Reward', 'counter'],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 30],
 [49, 14, 109.12507786313746, 1, 

In [25]:
q_table(3)

TypeError: 'numpy.ndarray' object is not callable

In [18]:
import numpy as np

# Define the state arrays
wind_speed_states_list = [0, 5, 10, 15, 20, 25, 30, 35, 40]
angle_states_list = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

# Define the action space
actions = [-30, 0, 30]

# Define the Q-table
q_table = np.zeros((len(wind_speed_states_list), len(angle_states_list), len(actions)))

# Define the Q-learning algorithm
def q_learning(current_wind_speed, current_angle, alpha, discount_factor, exploration_rate, num_episodes):
    for episode in range(num_episodes):
        # Find the closest index of the current wind speed and angle in the state arrays
        wind_speed_state = (np.abs(wind_speed_states_list - current_wind_speed)).argmin()
        angle_state = (np.abs(angle_states_list - current_angle)).argmin()
        
        # Choose an action using the exploration rate
        if np.random.uniform(0, 1) < exploration_rate:
            action = np.random.choice(actions)
        else:
            action = np.argmax(q_table[wind_speed_state][angle_state])
        
        # Update the Q-table
        next_wind_speed, next_angle = wind_model(current_wind_speed, current_angle, action)
        wind_speed_next_state = (np.abs(wind_speed_states_list - next_wind_speed)).argmin()
        angle_next_state = (np.abs(angle_states_list - next_angle)).argmin()
        reward = get_reward(next_wind_speed, next_angle)
        q_table[wind_speed_state][angle_state][action] = (1 - alpha) * q_table[wind_speed_state][angle_state][action] + alpha * (reward + discount_factor * np.max(q_table[wind_speed_next_state][angle_next_state]))
        
        # Update the current wind speed and angle
        current_wind_speed = next_wind_speed
        current_angle = next_angle

# Call the Q-learning function
q_learning(current_wind_speed, current_angle, alpha=0.1, discount_factor=0.95, exploration_rate=0.1, num_episodes=1000)

TypeError: unsupported operand type(s) for -: 'list' and 'int'